## Imports

In [ ]:
import sys
import os
repo_path = os.path.abspath('../')
sys.path.append(repo_path)
import scripts.addSA2 as addSA2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import geoplot.crs as gcrs
import geopandas as gpd
import geoplot as gplt
import pandas as pd
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


## Data Download

The first thing we need to do is download the datasources needed for our analysis.

This involves:
-> median income by SA2 level
-> population by SA2 level
-> SA2 Shape files
-> school location data
-> postcode/sa2 data
(see data_download.py in the script folder for details)

In [ ]:
%run ../scripts/data_download.py

## Scraping

For this project we decided to scrape rental data from domain.com.au.
This website has a huge number of listings, and by searching by postcode we collected a large number of metrics from each listed property.

This process is done in scrape.ipynb, and the results are saved in the raw data folder. as scraped_properties.csv as seen below.
(This will not be run in this notebook as it takes an extremely long time)

In [ ]:
scraped = pd.read_csv(f"../data/raw/scraped_properties.csv")
display(scraped.head(3))

## API Data

Now we get a bunch of distance data for various properties that we can merge into our other data

This is done with the school_proximities.ipynb notebook. But is quite an involved process, so that data is saved inside the git repository instead of being run here.

## Pre-Processing

While this gives us a lot of raw data, it needs to be put into a useable format first. This is done in pre_processing.py. By using a variety of regex rules to extract the relevant numbers from the data. We also need to remove a specific NSW town that was caught in the scrape for some reason. We also remove carspaces as they are not relevant to the problems.

From there we merge with the API distance data that has been gathered.

Now we can remove outliers. This is done mainly by plotting a linear regression model using:
-> number of beds
-> number of baths
-> number of parking spaces
-> distance to a school
-> distance to a park
-> distance to a shop
-> bond
and use this to predict rent price.

From this we reject values with a high cooks distance.
(See pre_processing.ipynb for proper details, as the .py file is purely for this notebook)

In [ ]:
%run ../scripts/pre_processing.py

Now our data looks like this:

In [ ]:
pre_processed = pd.read_csv(f"../data/curated/pre_processed_data.csv")
display(pre_processed.head(3))

We can also add in the SA2 zone and corresponding geometry for each of these listings. This allows us to visualise the data in various ways.

In [ ]:
sa2_grouped = addSA2.addSA2(pre_processed, use_postcode=False)


sa2_grouped = sa2_grouped.groupby("SA2").mean().reset_index()

shape = gpd.read_file('../data/raw/ShapeFile/SA2_2021_AUST_GDA2020.shp')
shape = shape.loc[shape.STE_NAME21 == "Victoria"]
shape = shape.loc[shape.geometry != None]
shape["SA2_CODE21"] = pd.to_numeric(shape["SA2_CODE21"], errors='ignore')
sa2_grouped = gpd.GeoDataFrame(sa2_grouped.join(shape.set_index("SA2_CODE21")["geometry"], on="SA2"))
sa2_grouped = sa2_grouped[sa2_grouped["geometry"] != None]

If you want to see a plot of any feature, simply type the feature into the "FEATURE HERE" section below:

In [ ]:
# Plot the lines for the visualisation
ax = gplt.polyplot(shape, figsize=(100, 50))
# create a heatmap based on a particular feature
gplt.choropleth(
  sa2_grouped,
  hue="count",
  edgecolor="black",
  linewidth=0.1,
  cmap="Greens",
  legend=True,
  ax=ax
)

plt.savefig("counts.jpg")

## Question 1

### Read in data

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

DIR_CUR = "../data/curated/"
DIR_PLT = "../plots/"

In [ ]:
# read in suburb level version of data
df = pd.read_csv(f"{DIR_CUR}suburb_data.csv")
display(df.head(5))

## Most important data distributions

In [ ]:
def distribution(x, bins, x_lab):
    plt.hist(x, bins=bins, density=True)
    plt.title(f"{x_lab} Distribution", size=20)
    plt.xlabel(x_lab, size=16)
    plt.ylabel("Frequency", size=16)
    plt.show()
    
distribution(df["weekly_rent"], 30, "Weekly Rent (AUS)", )
#distribution(df["bond"], 30, "Bond (AUS)")
distribution(df["school_distance"], 30, "School Distance (m)")
distribution(df["park_distance"], 30, "Park Distance (m)")
distribution(df["shop_distance"], 30, "Shop Distance (m)")
distribution(df["train_distance"], 30, "Train Distance (m)")
distribution(df["stop_distance"], 30, "Stop Distance (m)")

## VS plots
(uncomment the variable you want to see)

In [ ]:
def vs(x, y, x_lab, y_lab, x_trans=lambda x: x, y_trans=lambda x: x):
    plt.scatter(x_trans(x), y_trans(y))
    plt.title(f"{y_lab} vs {x_lab}", size=20)
    plt.xlabel(x_lab, size=16)
    plt.ylabel(y_lab, size=16)
    plt.show()

vs(df["school_distance"], df["weekly_rent"], "School Distance (m)", "Weekly Rent (AUS)")
#vs(df["park_distance"],   df["weekly_rent"], "Park Distance (m)",   "Weekly Rent (AUS)")
#vs(df["shop_distance"],   df["weekly_rent"], "Shop Distance (m)",   "Weekly Rent (AUS)")
#vs(df["train_distance"],  df["weekly_rent"], "Train Distance (m)",  "Weekly Rent (AUS)")
#vs(df["stop_distance"],   df["weekly_rent"], "Stop Distance (m)",   "Weekly Rent (AUS)")

#vs(df["population"], df["weekly_rent"], "Population", "Weekly Rent (AUS)")

#vs(df["count"], df["weekly_rent"], "_", "Weekly Rent (AUS)")

#vs(df["performance_avg_days_on_market"], df["weekly_rent"], "_", "Weekly Rent (AUS)")
#vs(df["median_weekly_income"], df["weekly_rent"], "median weekly income", "Weekly Rent (AUS)")
#vs(df["performance_median_price"], df["weekly_rent"], "Median Price", "Weekly Rent (AUS)")

Cross validation used in full model, however not used here as it takes longer to run.

In [ ]:
response = "weekly_rent"
predictors = ["median_weekly_income", "performance_median_price"] 

df_na_out = df.dropna(subset=predictors)

trans = ColumnTransformer(transformers=[("scale", StandardScaler(), predictors)],
                          remainder="passthrough") 

x_train, x_test, y_train, y_test = train_test_split(df_na_out[predictors],
                                                    df_na_out[response],
                                                    test_size=0.20,
                                                    random_state=30027)

x_train_trans = trans.fit_transform(x_train)
x_test_trans = trans.fit_transform(x_test)

model = MLPRegressor(random_state=30027, max_iter=1000000, n_iter_no_change=15).fit(x_train_trans, y_train)
model.score(x_test_trans, y_test)

In [ ]:
from sklearn.linear_model import LinearRegression


model_2 = LinearRegression().fit(x_train_trans, y_train)
model_2.score(x_test_trans, y_test)

## Question 2

For this question we need historical data instead of current, data. This has been scraped from domain.com.au, and is pre-processed seperately. (See predict_future_preprocess.ipynb for details)

In [ ]:
%run ../scripts/predict_future_preprocessing.py

In [ ]:
# We read in the dataframe for predictions
df = pd.read_csv(f"../data/curated/future_prediction_data.csv")

In [ ]:
display(df)

In [ ]:
# For the linear model, exclude 0 clearance rate and no new building locations.
df = df[df["clearance"] != 0]
df = df[df["new_dwellings_2019"] != 0]

In [ ]:
X = df[["2019_n_sold", "suburb_population", "new_dwellings_2019", "non_residential_value_2019"]]
y = df["3_year_growth"]

In [ ]:
# Fit a linear model to the data
reg = LinearRegression().fit(X,y)
reg.score(X,y)

Clearly, using a linear regression on historical data is not a valid method, so we will have to try another method to predict future value.
Instead we will have to take a non-statistical approach, and use the suburb metrics that we have access to.

## Predictor Metrics:

Historical 3-year-growth: While a regression to the mean is fairly likely in many cases, a suburb having a high growth rate in previous years should still indicate that that is more likely to continue.

Current growth rate: Extrapolating the 2022 growth rate to a 3 year prediction is unlikely to be accurate, but it is once again a decent indicator.

New dwellings / population: If there is a high number of new dwellings relative to the population, then in theory the increased supply will lead to decreased house value.

non residential value: The theory behind using this metric, is that non-residential housing indicates an increase in local business. This should in theory increase the value of the area. This may be incorrect, as it might indicate a non-residential area, but not sure.

Clearance: A high clearance rate should indicate a high demand in the area, so in theory an increase in future prices.

Average days on market: A low number of days on the market, should indicate (like clearance), a high demand for houses in the area.

sold / population: If there are a high number of sales relative to the population it indicates market interest.

Unfortunately due to a lack of historical data for many areas, and many of these metrics, we will have to decide aribtrarily on the importance of each feature. I will then create a ranking of each feature and use this as the input for the algorithm.

https://propertyupdate.com.au/property-investment-melbourne/#is-it-the-right-time-to-get-into-the-melbourne-property-market

https://www.trilogyfunding.com.au/blog/7-key-market-indicators-every-property-investor-should-understand-april-2015/


https://www.mmj.com.au/resources/blog/5-key-market-indicators-every-property-investor-should-know/

In [ ]:
df = pd.read_csv(f"../data/curated/future_prediction_data.csv")

# Get rankings for each feature
predict_df = pd.DataFrame()
predict_df["suburb"] = df["suburb"]
predict_df["avg_days_on_market"] = df["avg_days_on_market"].rank(ascending = False) # low is better
predict_df["3_year_growth"] = df["3_year_growth"].rank(ascending = True) # high is better
predict_df["2022_growth"] = df["2022_growth"].rank(ascending = True) # high is better
predict_df["sold/pop"] = (df["2022_n_sold"]/df["suburb_population"]).rank(ascending = True).fillna(0) # high is better
predict_df["dwellings/pop"] = (df["new_dwellings_2021"]/df["suburb_population"]).rank(ascending = False).fillna(0) # low is better
predict_df["non_residential_value"] = df["non_residential_value_2021"].rank(ascending = True) # higher is better
predict_df["clearance"] = df["clearance"].rank(ascending = True) # higher is better
predict_df["sum"] = predict_df.sum(axis=1)
predict_df["SA2"] = df["SA2"]

In [ ]:
predict_df = predict_df.sort_values(by="sum", ascending=False)
predict_df.to_csv(f"../data/curated/prediction_results.csv")

In [ ]:
display(predict_df)

We can then see the ranking of suburbs. Flinders, Blairgowrie, tootgarook, mccrae, and mornington take the top spots.

In [ ]:
shape = gpd.read_file('../data/raw/ShapeFile/SA2_2021_AUST_GDA2020.shp')
shape = shape.loc[shape.STE_NAME21 == "Victoria"]
shape = shape.loc[shape.geometry != None]
shape["SA2_CODE21"] = pd.to_numeric(shape["SA2_CODE21"], errors='ignore')
plot_df = predict_df.join(shape[["SA2_CODE21", "geometry"]].set_index("SA2_CODE21"), on="SA2")
plot_df = plot_df[plot_df["geometry"] != None]

In [ ]:
from matplotlib.pyplot import figure
import geoplot.crs as gcrs


ax = gplt.polyplot(shape, figsize=(100, 50))#, projection=gcrs.AlbersEqualArea())

gplt.choropleth(
  gpd.GeoDataFrame(plot_df),
  hue="sum",
  edgecolor="black",
  linewidth=0.1,
  cmap="Greens",
  legend=True,
  ax=ax
)

This graphic shows the results per SA2 zone, but because they don't line up with the suburbs very well, so we get a lot of missing values.

## Question 3

In [ ]:
df = pd.read_csv(f"../data/curated/suburb_data.csv")
df